In [1]:
import sqlite3

In [2]:
! mkdir -p databases

In [3]:
DB_PATH = "databases/test3.db"

In [4]:
connection = sqlite3.connect(DB_PATH)

## Insert

### Pure SQL Approach

In [5]:
cursor = connection.cursor()

cursor.execute('''
    CREATE TABLE Purchase(
        date TEXT,
        trans TEXT,
        symbol TEXT,
        qty REAL,
        price REAL
    );
''')

cursor.execute('''
    INSERT INTO Purchase VALUES
        ('2006-01-05', 'BUY', 'RHAT', 100, 35.14),
        ('2006-03-28', 'BUY', 'IBM', 1000, 45.00),
        ('2006-04-05', 'BUY', 'MSFT', 1000, 72.00),
        ('2006-04-06', 'SELL', 'IBM', 500, 53.00);
''')

connection.commit()
connection.close()

### "Programmer's" Approach

In [6]:
connection = sqlite3.connect(DB_PATH)
cursor = connection.cursor()

purchases = [
    ('2022-03-03', 'BUY', 'IKEA', 10, 50.00),
    ('2022-03-09', 'BUY', 'McDonald\'s', 20, 20.00),
    ('2022-03-10', 'BUY', 'Coca-Cola', 100, 30.00),
]

cursor.executemany(
    "INSERT INTO Purchase VALUES (?, ?, ?, ?, ?);",
    purchases
)

connection.commit()
connection.close()

## Select

In [7]:
connection = sqlite3.connect(DB_PATH)
cursor = connection.cursor()

cursor.execute("SELECT * FROM Purchase WHERE symbol='RHAT';")

print(cursor.fetchone())

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)


In [8]:
print(cursor.fetchone())

None


In [9]:
cursor.execute("SELECT * FROM Purchase WHERE symbol='RHAT';")

print(cursor.fetchmany())

[('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)]


In [10]:
for row in cursor.execute("SELECT * FROM Purchase ORDER BY price;"):
    print(row)

('2022-03-09', 'BUY', "McDonald's", 20.0, 20.0)
('2022-03-10', 'BUY', 'Coca-Cola', 100.0, 30.0)
('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)
('2006-03-28', 'BUY', 'IBM', 1000.0, 45.0)
('2022-03-03', 'BUY', 'IKEA', 10.0, 50.0)
('2006-04-06', 'SELL', 'IBM', 500.0, 53.0)
('2006-04-05', 'BUY', 'MSFT', 1000.0, 72.0)


In [11]:
connection.close()

## Update, Delete

In [12]:
connection = sqlite3.connect(DB_PATH)
cursor = connection.cursor()

for row in cursor.execute("SELECT * FROM Purchase;"):
    print(row)

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)
('2006-03-28', 'BUY', 'IBM', 1000.0, 45.0)
('2006-04-05', 'BUY', 'MSFT', 1000.0, 72.0)
('2006-04-06', 'SELL', 'IBM', 500.0, 53.0)
('2022-03-03', 'BUY', 'IKEA', 10.0, 50.0)
('2022-03-09', 'BUY', "McDonald's", 20.0, 20.0)
('2022-03-10', 'BUY', 'Coca-Cola', 100.0, 30.0)


In [13]:
def select_all(connection, table_name: str) -> None:
    cursor = connection.cursor()

    for row in cursor.execute(f"SELECT * FROM {table_name};"):
        print(row)

In [14]:
column_names = [
    description[0] for description in cursor.description
]

In [15]:
print(column_names)

['date', 'trans', 'symbol', 'qty', 'price']


### Update and Rollback

In [16]:
cursor.execute(
    "UPDATE Purchase SET price = price * 2 WHERE symbol = 'IBM';"
)

In [29]:
select_all(connection, 'Purchase')

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)
('2006-03-28', 'BUY', 'IBM', 1000.0, 90.0)
('2006-04-05', 'BUY', 'MSFT', 1000.0, 72.0)
('2006-04-06', 'SELL', 'IBM', 500.0, 106.0)
('2022-03-03', 'BUY', 'IKEA', 10.0, 50.0)
('2022-03-09', 'BUY', "McDonald's", 20.0, 20.0)
('2022-03-10', 'BUY', 'Coca-Cola', 100.0, 30.0)


Another connection does not see changes if they are not committed.

In [30]:
connection2 = sqlite3.connect(DB_PATH)
cursor2 = connection2.cursor()

for row in cursor2.execute("SELECT * FROM Purchase;"):
    print(row)

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)
('2006-03-28', 'BUY', 'IBM', 1000.0, 45.0)
('2006-04-05', 'BUY', 'MSFT', 1000.0, 72.0)
('2006-04-06', 'SELL', 'IBM', 500.0, 53.0)
('2022-03-03', 'BUY', 'IKEA', 10.0, 50.0)
('2022-03-09', 'BUY', "McDonald's", 20.0, 20.0)
('2022-03-10', 'BUY', 'Coca-Cola', 100.0, 30.0)


In [32]:
for row in cursor2.execute("SELECT * FROM Purchase;"):
    print(row)

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)
('2006-03-28', 'BUY', 'IBM', 1000.0, 90.0)
('2006-04-05', 'BUY', 'MSFT', 1000.0, 72.0)
('2006-04-06', 'SELL', 'IBM', 500.0, 106.0)
('2022-03-03', 'BUY', 'IKEA', 10.0, 50.0)
('2022-03-09', 'BUY', "McDonald's", 20.0, 20.0)
('2022-03-10', 'BUY', 'Coca-Cola', 100.0, 30.0)


In [18]:
connection.rollback()

In [19]:
select_all(connection, 'Purchase')

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)
('2006-03-28', 'BUY', 'IBM', 1000.0, 45.0)
('2006-04-05', 'BUY', 'MSFT', 1000.0, 72.0)
('2006-04-06', 'SELL', 'IBM', 500.0, 53.0)
('2022-03-03', 'BUY', 'IKEA', 10.0, 50.0)
('2022-03-09', 'BUY', "McDonald's", 20.0, 20.0)
('2022-03-10', 'BUY', 'Coca-Cola', 100.0, 30.0)


### Update, Delete, and Rollback

In [26]:
cursor.execute(
    "UPDATE Purchase SET price = price * 2 WHERE symbol = 'IBM';"
)

In [27]:
select_all(connection, 'Purchase')

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)
('2006-03-28', 'BUY', 'IBM', 1000.0, 90.0)
('2006-04-05', 'BUY', 'MSFT', 1000.0, 72.0)
('2006-04-06', 'SELL', 'IBM', 500.0, 106.0)
('2022-03-03', 'BUY', 'IKEA', 10.0, 50.0)
('2022-03-09', 'BUY', "McDonald's", 20.0, 20.0)
('2022-03-10', 'BUY', 'Coca-Cola', 100.0, 30.0)


In [22]:
cursor.execute("DELETE FROM Purchase WHERE symbol = 'RHAT';")

In [23]:
select_all(connection, 'Purchase')

('2006-03-28', 'BUY', 'IBM', 1000.0, 90.0)
('2006-04-05', 'BUY', 'MSFT', 1000.0, 72.0)
('2006-04-06', 'SELL', 'IBM', 500.0, 106.0)
('2022-03-03', 'BUY', 'IKEA', 10.0, 50.0)
('2022-03-09', 'BUY', "McDonald's", 20.0, 20.0)
('2022-03-10', 'BUY', 'Coca-Cola', 100.0, 30.0)


In [24]:
connection.rollback()

In [25]:
for row in cursor.execute("SELECT * FROM Purchase;"):
    print(row)

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)
('2006-03-28', 'BUY', 'IBM', 1000.0, 45.0)
('2006-04-05', 'BUY', 'MSFT', 1000.0, 72.0)
('2006-04-06', 'SELL', 'IBM', 500.0, 53.0)
('2022-03-03', 'BUY', 'IKEA', 10.0, 50.0)
('2022-03-09', 'BUY', "McDonald's", 20.0, 20.0)
('2022-03-10', 'BUY', 'Coca-Cola', 100.0, 30.0)


In [33]:
connection.close()

## Context Manager To Do Things Without Commit/Rollback

In [34]:
connection = sqlite3.connect(":memory:")

connection.execute("""
    CREATE TABLE Person(
        id INTEGER PRIMARY KEY,
        firstname VARCHAR UNIQUE
    );
""")


with connection:
    connection.execute(
        "INSERT INTO Person(firstname) VALUES (?);", ("Joe",)
    )


try:
    with connection:
        connection.execute(
            "INSERT INTO Person(firstname) VALUES (?);", ("Joe",)
        )
except sqlite3.IntegrityError:
    print("Can't add Joe twice!")


connection.close()

Can't add Joe twice!


## "Programmer's" SELECT

In [35]:
connection = sqlite3.connect(DB_PATH)
connection.row_factory = sqlite3.Row

cursor = connection.cursor()

cursor.execute("SELECT * FROM Purchase WHERE symbol='RHAT';")
row = cursor.fetchone()

In [36]:
row

In [37]:
print(row.keys())

['date', 'trans', 'symbol', 'qty', 'price']


In [38]:
for key in row.keys():
    print(row[key])

2006-01-05
BUY
RHAT
100.0
35.14


In [39]:
dict(row)

{'date': '2006-01-05',
 'trans': 'BUY',
 'symbol': 'RHAT',
 'qty': 100.0,
 'price': 35.14}

In [41]:
# https://docs.python.org/3/library/sqlite3.html#sqlite3.Connection.row_factory

class DictFactory:
    def __call__(self, cursor, row):
        result = dict()
        
        for index, column_data in enumerate(cursor.description):
            column_name = column_data[0]
            result[column_name] = row[index]
        
        return result

In [42]:
connection.row_factory = DictFactory()

cursor = connection.cursor()

cursor.execute("SELECT * FROM Purchase WHERE symbol='RHAT';")
row = cursor.fetchone()

In [43]:
row

{'date': '2006-01-05',
 'trans': 'BUY',
 'symbol': 'RHAT',
 'qty': 100.0,
 'price': 35.14}

In [44]:
connection.close()

# PyMongo

https://pymongo.readthedocs.io/en/stable/tutorial.html

In [1]:
from pymongo import MongoClient

```bash
# Install MongoDB ...

sudo mongod
```

![](./images/MongoRunning0.jpg)
![](./images/MongoRunning.jpg)

In [3]:
client = MongoClient('localhost', 27017)

In [4]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [5]:
db = client.test_database

In [6]:
collection = db.test_collection

In [7]:
collection.insert_one({
    'name': 'Dopey',
    'race': 'dwarf'
})

In [8]:
db.list_collection_names()

['test_collection']

In [9]:
collection.insert_one({
    'name': 'Legolas',
    'race': 'elf'
})

In [10]:
collection.find_one()

{'_id': ObjectId('622c68f1295d2bf57b80c552'), 'name': 'Dopey', 'race': 'dwarf'}

In [11]:
collection.find_one({'race': 'elf'})

{'_id': ObjectId('622c6934295d2bf57b80c553'),
 'name': 'Legolas',
 'race': 'elf',
 'age': 3000}

In [12]:
collection.count_documents({})

4

In [13]:
collection.update_one(
    {'name': 'Legolas'},
    {'$set': {'age': 3000}}
)

In [14]:
collection.find_one({'race': 'elf'})

{'_id': ObjectId('622c6934295d2bf57b80c553'),
 'name': 'Legolas',
 'race': 'elf',
 'age': 3000}